In [16]:
import numpy as np 
from numpy import linalg as LA
import pandas as pd 
from matplotlib import pyplot as plt 
from numba import jit
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error



""" Angular Momentum Operators """
@jit
def Jz(bra,ket):
    m = float(ket)
    if bra == ket:
        return m
    else: 
        return 0
@jit
def JJz(J_power,Jz_power,ket):     # For Jz^a[J(J+1)]^b
    j=3.5
    m=float(ket)
    return(((j*(j+1))**J_power)*(m**Jz_power))

@jit
def Jp(power,bra,ket):
    j=3.5
    ket=float(ket)
    bra=float(bra)
    m=ket
    jp=1
    
    if (ket+power == bra):
        for i in range(power):
            jp*= np.sqrt( j*(j+1) - (m+i)*(m+1+i) )
            ket+=1
    else: 
        jp=0
    
    return(jp,ket) 
@jit   
def Jm(power,bra,ket):
    j=3.5
    ket=float(ket)
    bra=float(bra)
    m=ket
    
    jm=1
    
    if (ket-power == bra):
        for i in range(power):
            jm*= np.sqrt( j*(j+1) - (m-i)*(m-1-i) )
            ket-=1
    else: 
        jm=0
    
    return(jm,ket) 

""" Steven's Operator functions """
@jit
def S_02(bra,ket): 
    s=3*JJz(0,2,ket)-JJz(1,0,ket)
    
    return s*(int(bra==ket))

@jit
def S_04(bra,ket):
    s=35*JJz(0,4,ket)-30*JJz(1,2,ket)+25*JJz(0,2,ket)-6*JJz(1,0,ket)+3*JJz(2,0,ket)
    return(s*(int(bra==ket)))

@jit
def S_06(bra,ket):
    s=231*JJz(0,6,ket)-315*JJz(1,4,ket)+735*JJz(0,4,ket)+105*JJz(2,2,ket)-525*JJz(1,2,ket)+294*JJz(0,2,ket)-5*JJz(3,0,ket)+40*JJz(2,0,ket)-60*JJz(1,0,ket)
    return(s*(int(bra==ket)))

@jit
def S_66(bra,ket):
    return( 0.5*(Jp(6,bra,ket)[0]+Jm(6,bra,ket)[0])) 

@jit
def S_34(bra,ket):
    
    a=JJz(0,1,Jp(3,bra,ket)[1])*Jp(3,bra,ket)[0] + JJz(0,1,Jm(3,bra,ket)[1])*Jm(3,bra,ket)[0]   
    a+= JJz(0,1,ket)*(Jp(3,bra,JJz(0,1,ket))[0] + Jm(3,bra,JJz(0,1,ket))[0])
    
    return 0.25*a

@jit
def S_36(bra,ket): 
    
    a=(11*JJz(0,3,ket)-3*JJz(1,1,ket)-59*JJz(0,1,ket))*(Jp(3,bra,ket)[0]+Jm(3,bra,ket)[0])
    
    ket_p=Jp(3,bra,ket)[1]
    ket_m=Jm(3,bra,ket)[1]
    
    a+=Jp(3,bra,ket)[0]*((11*JJz(0,3,ket_p)-3*JJz(1,1,ket_p)-59*JJz(0,1,ket_p))) 
    a+=Jm(3,bra,ket)[0]*((11*JJz(0,3,ket_m)-3*JJz(1,1,ket_m)-59*JJz(0,1,ket_m)))
    
    return a*0.25


""" Hamiltonian Calculation from Steven's operator parameters """

#@jit
def Matrix(B): 
    M=np.zeros( [8,8], dtype=float)    
    print(type(M))
    #B=[1.135,-0.0615,0.0011,0.005,0.315,0.037]
    
    for i in range(8):
        for j in range(8):
            ket=i-3.5
            bra=j-3.5
            if i==j:
                M[i][j]= B[0]*S_02(bra,ket)+B[1]*S_04(bra,ket)+B[2]*S_06(bra,ket)
            else:
                M[i][j]= B[3]*S_66(bra,ket)+B[4]*S_34(bra,ket)+B[5]*S_36(bra,ket)  
    return(M)
            


""" Eigen System Calculation  """ 
@jit
def eigenfunc(a):
    return(np.linalg.eigh(a))

""" Error metric 1, dont use for minimisation """

def err(B,r=False):
    #B = [1.135,-0.0615,0.0011,0.005,0.315,0.037]
    actual = [0,76.7989352,116.24306695,0.,76.7989352 , 116.24306695,  82.11933405,  82.11933405]
    actual = np.sort(actual)

    temp = eigenfunc(Matrix(B))
    a = temp[0]
    a = np.sort(a)
    a -= a[0]
    
    rms = (mean_squared_error(a, actual))+ mean_absolute_error(a,actual)
    #return [rms,B,a] 
    if r==False:
        return rms
    else:
        return [rms,B,a,temp]
    
    
""" Optimised SQW to be used with np.eigh() not eig(), Input:: takes in eigensystem object """
#@jit    
def Sqw_optim(temp):
    
    a = temp[0]
    M = temp[1]
    
    a_0 = M[:,0]
    b_0 = M[:,1]

    a_1 = M[:,2]
    b_1 = M[:,3]

    a_2 = M[:,4]
    b_2 = M[:,5]

    a_3 = M[:,6]
    b_3 = M[:,7]
    
    
    
    i1 = Intensities(a_3,b_3,a_0,b_0)/Intensities(a_2,b_2,a_0,b_0)
    i2 = Intensities(a_1,b_1,a_0,b_0)/Intensities(a_2,b_2,a_0,b_0)
    

    return np.sort([i2,i1])

""" Eigenstate as input, use with np.eigh() """

def Sqw(temp):
    
    a = temp[0]
    M = temp[1]
    b = a
    a=np.sort(a)  
    z=np.where(np.isclose(b,a[0])),np.where(np.isclose(b,a[2])),np.where(np.isclose(b,a[4])),np.where(np.isclose(b,a[6]))
    
    a_0 = M[:,z[0][0][0]]
    b_0 = M[:,z[0][0][1]]

    a_1 = M[:,z[1][0][0]]
    b_1 = M[:,z[1][0][1]]

    a_2 = M[:,z[2][0][0]]
    b_2 = M[:,z[2][0][1]]

    a_3 = M[:,z[3][0][0]]
    b_3 = M[:,z[3][0][1]]
    
    
    
    i1 = Intensities(a_3,b_3,a_0,b_0)/Intensities(a_2,b_2,a_0,b_0)
    i2 = Intensities(a_1,b_1,a_0,b_0)/Intensities(a_2,b_2,a_0,b_0)
    

    return np.sort([i2,i1])




""" Steven parameter as input, not optimised """



def Sqw2(B):
    temp = eigenfunc(Matrix(B))
    a = temp[0]
    M = temp[1]
    b = a
    a=np.sort(a)  
    z=np.where(np.isclose(b,a[0])),np.where(np.isclose(b,a[2])),np.where(np.isclose(b,a[4])),np.where(np.isclose(b,a[6]))
    
    a_0 = np.round(M[:,z[0][0][0]],5)
    
    b_0 = np.round(M[:,z[0][0][1]],5)

    a_1 = np.round(M[:,z[1][0][0]],5)
    b_1 = np.round(M[:,z[1][0][1]],5)

    a_2 = np.round(M[:,z[2][0][0]],5)
    b_2 = np.round(M[:,z[2][0][1]],5)

    a_3 = np.round(M[:,z[3][0][0]],5)
    b_3 = np.round(M[:,z[3][0][1]],5)
    
    
    
    i1 = Intensities(a_3,b_3,a_0,b_0)/Intensities(a_2,b_2,a_0,b_0)
    i2 = Intensities(a_1,b_1,a_0,b_0)/Intensities(a_2,b_2,a_0,b_0)
    

    return np.sort([i2,i1])



""" Input (excitation doublet , ground states doublet) """ 
@jit
def Intensities(a_1,b_1,a_0,b_0):
    sum =0
    jx=0
    ijy=0
    jz=0
    for i in np.arange(8):
        for j in np.arange(8):
            jx += (Jm(1,3.5-i,3.5-j)[0]+Jp(1,3.5-i,3.5-j)[0])*a_1[j]*a_0[i]/2
            ijy += (-Jm(1,3.5-i,3.5-j)[0]+Jp(1,3.5-i,3.5-j)[0])*a_1[j]*a_0[i]/2
            jz += Jz(3.5-i,3.5-j)*a_1[j]*a_0[i]
            
    sum = jx**2 + (ijy)**2 + jz**2
    
    jx=0
    ijy=0
    jz=0
    for i in np.arange(8):
        for j in np.arange(8):
            jx += (Jm(1,3.5-i,3.5-j)[0]+Jp(1,3.5-i,3.5-j)[0])*b_1[j]*a_0[i]/2
            ijy += (-Jm(1,3.5-i,3.5-j)[0]+Jp(1,3.5-i,3.5-j)[0])*b_1[j]*a_0[i]/2
            jz += Jz(3.5-i,3.5-j)*b_1[j]*a_0[i]
            
    sum += jx**2 + (ijy)**2 + jz**2
    
    jx=0
    ijy=0
    jz=0
    for i in np.arange(8):
        for j in np.arange(8):
            jx += (Jm(1,3.5-i,3.5-j)[0]+Jp(1,3.5-i,3.5-j)[0])*a_1[j]*b_0[i]/2
            ijy += (-Jm(1,3.5-i,3.5-j)[0]+Jp(1,3.5-i,3.5-j)[0])*a_1[j]*b_0[i]/2
            jz += Jz(3.5-i,3.5-j)*a_1[j]*b_0[i]
            
    sum += jx**2 + (ijy)**2 + jz**2 
    jx=0
    ijy=0
    jz=0
    for i in np.arange(8):
        for j in np.arange(8):
            jx += (Jm(1,3.5-i,3.5-j)[0]+Jp(1,3.5-i,3.5-j)[0])*b_1[j]*b_0[i]/2
            ijy += (-Jm(1,3.5-i,3.5-j)[0]+Jp(1,3.5-i,3.5-j)[0])*b_1[j]*b_0[i]/2
            jz += Jz(3.5-i,3.5-j)*b_1[j]*b_0[i]
            
    sum += jx**2 + (ijy)**2 + jz**2
            
            
            
    return (sum)
    
    
    

    
""" Error Metric 2 """


@jit
def err2(B):
    
    eigensystem = eigenfunc(Matrix(B))
    
    calc_rel_intensity = np.array(Sqw_optim(eigensystem))*100
    
    er = 10*B[0]
    
    temp2 = eigensystem
    a = temp2[0]
    a = np.sort(a)
    a -= a[0]
    calc_energies = a
    
    return (np.sqrt(mean_squared_error(calc_rel_intensity,expt_rel_intensity) 
                    + mean_squared_error(calc_energies, expt_energies) 
                     + mean_squared_error(B,[1.270,-0.0372,0.00025,0.0024,0.275,0.0023])) 
                    + mean_absolute_error(calc_energies, expt_energies))   


#+ mean_squared_error(B,[1.270,-0.0372,0.00025,0.0024,0.275,0.0023])


""" Reducing calculation cost """


B=[1.135,-0.0615,0.0011,0.005,0.315,0.037]
actual = eigenfunc(Matrix(B))[0]
#actual = [0,76.706,116.23,0.,76.706 , 116.23,  81.764,  82.764]
actual = np.sort(actual)  
actual = actual -actual[0]
expt_energies = actual
expt_rel_intensity = np.sort(np.array(Sqw_optim(eigenfunc(Matrix(B)))))*100


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [10]:
""" Minimisation using randomised algorithm : Basin Hopping, Use with  Powell minimizer """


from scipy.optimize import basinhopping 

def fit(paper = [1.270,-0.0372,0.00025,0.0024,0.275,0.0023], number_of_iterations=5):
#b =[1.135,-0.0615,0.0011,0.005,0.315,0.037] +  0.5*np.ones(6)
    b = paper
    return basinhopping(err2, b, niter = number_of_iterations, minimizer_kwargs ={ 'method' : 'Powell'})

In [11]:
from IPython.display import clear_output

# calling the fit function with initial parameters. 
result = fit(number_of_iterations = 30)
clear_output()


In [17]:
from IPython.display import clear_output

error = err2(result.x) # gives warning so for clearing output put this in a seperate line
clear_output()
print("\n Parameters Extracted From Fit :: ")
print(np.around(result.x,5))#, result.success, result.message)
print('\n Error(Parameters)')
print(np.sqrt(mean_squared_error(result.x,[1.135,-0.0615,0.0011,0.005,0.315,0.037])))
print('\n Total  Error = ')
#print(err2(result.x), err2([1.135,-0.0615,0.0011,0.005,0.315,0.037]))
print(error)



res = eigenfunc(Matrix(result.x))
t= res[0] 
t =t - min(t)
print("Calculated Eigen Energies")
print(np.round(t,4))
print("Jonathan's Eigen Energies")
print(actual)
print("Calculated Relative Intensity \n")
print(Sqw2(result.x))

print("Error in Relative Intensity :: ",mean_squared_error(expt,Sqw2(result.x)))


print("\n")
#print(np.around(res[1][:,0],3),res[0][0])

gs_1 = np.round(eigenfunc(Matrix(result.x))[1][:,0],3)
gs_2 = np.round(eigenfunc(Matrix(result.x))[1][:,1],3)
print("Ground State anisotropy")
print(np.round(2*jz(gs_1)/jpp(gs_1,gs_2),1))



print("Diagonalised Hamiltonian")
print("\n")


for i in range(8):
        
    print( str(-i+3.5) + str(np.around(res[1][i,:],3)))
    print("\n")

print("\n")

print("Jonathan's Hamiltonian")
p = eigenfunc(Matrix([1.135,-0.0615,0.0011,0.005,0.315,0.037]))[1]
for i in range(8):
    
    print(str(-i+3.5) + str(np.around(p[i,:],3)))
    

#[1.135,-0.0615,0.0011,0.005,0.315,0.037]


 Parameters Extracted From Fit :: 
[ 1.20555e+00 -6.11700e-02  1.16000e-03 -4.74000e-03 -1.27860e-01
 -3.99200e-02]

 Error(Parameters)
0.18579412056226247

 Total  Error = 
0.16893190758994198
<class 'numpy.ndarray'>
Calculated Eigen Energies
[  0.       0.      76.7989  76.7989  82.1193  82.1193 116.245  116.245 ]
Jonathan's Eigen Energies
[0.00000000e+00 4.26325641e-14 7.67989352e+01 7.67989352e+01
 8.21193340e+01 8.21193340e+01 1.16243067e+02 1.16243067e+02]
Calculated Relative Intensity 

<class 'numpy.ndarray'>
[0.19023728 0.28426084]
<class 'numpy.ndarray'>
Error in Relative Intensity ::  574.011298939931


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Ground State anisotropy
-0.4
Diagonalised Hamiltonian


3.5[ 0.3    0.018 -0.01   0.954 -0.     0.    -0.015  0.001]


2.5[-0.01   0.17  -0.038 -0.    -0.    -0.    -0.072 -0.982]


1.5[-0.  0.  0. -0. -1.  0.  0.  0.]


0.5[-0.937 -0.057 -0.003  0.298  0.     0.     0.174 -0.013]


-0.5[-0.057  0.937 -0.298 -0.003 -0.     0.  

In [14]:
gs_1 = np.round(eigenfunc(Matrix(B))[1][:,0],3)
gs_2 = np.round(eigenfunc(Matrix(B))[1][:,1],3)

2*jz(gs_1)/jpp(gs_1,gs_2)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


0.5372344711447377

In [7]:
matrix = Matrix([1.135,-0.0615,0.0011,0.005,0.315,0.037])
eigen_system = eigenfunc(matrix)

eigen_energy = np.round(eigen_system[0]-eigen_system[0][0],3) 

eigen_vector = eigen_system[1]
#gs_1 = np.round(eigen_vector[:,0],3)
#gs_2 = np.round(eigen_vector[:,1],3)




def jz(gs_1):
    result = 0
    for i in range(8):
        m = 3.5-i
        result += m*gs_1[i]**2
    return result

def jpp(gs_1, gs_2):
    J = 3.5
    temp = np.zeros(8)
    for i in range(7):
        m = 3.5 - i
        temp[i+1] = np.sqrt(J*(J+1)-m*(m+1))*gs_1[i]
    gs_1 = temp
    return(np.dot(gs_1,gs_2))

2*jz(gs_1)/jpp(gs_1,gs_2)


<class 'numpy.ndarray'>


/Users/suvam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in double_scalars


-inf

In [54]:

gs_1 = np.round(eigenfunc(Matrix(result.x))[1][:,0],3)
gs_2 = np.round(eigenfunc(Matrix(result.x))[1][:,1],3)

2*jz(gs_1)/jpp(gs_1,gs_2)



<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[ 0.          0.          0.          0.         -1.52595544 -0.
  0.          1.01151767] [-0.    -0.292  0.     0.    -0.394  0.    -0.    -0.871]


-18.010381904710226

In [24]:
print('error')
print(err2(result.x))
      
print("Relative Intensity \n")
print(Sqw2(result.x), mean_squared_error(expt,Sqw2(result.x)))


print("\n")
#print(np.around(res[1][:,0],3),res[0][0])
print("\n")


for i in range(8):
        
    print( str(-i+3.5) + str(np.around(res[1][i,:],3)))
    print("\n")

print("\n")

p = eigenfunc(Matrix([1.135,-0.0615,0.0011,0.005,0.315,0.037]))[1]
for i in range(8):
    
    print(str(-i+3.5) + str(np.around(p[i,:],3)))
    

#[1.135,-0.0615,0.0011,0.005,0.315,0.037]

error
<class 'numpy.ndarray'>
1.1676518252584003e-12
Relative Intensity 

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[0.01991201 0.02448274] 4.8807282202585665




3.5[-0.015 -0.117  0.009  0.992  0.     0.    -0.009 -0.047]


2.5[-0.136  0.018 -0.063  0.001 -0.    -0.     0.972 -0.181]


1.5[-0.  0. -0.  0. -0.  1.  0.  0.]


0.5[-0.126 -0.975 -0.001 -0.11  -0.    -0.     0.026  0.141]


-0.5[-0.975  0.126 -0.11   0.001  0.     0.    -0.141  0.026]


-1.5[ 0.  0.  0. -0.  1.  0.  0. -0.]


-2.5[ 0.018  0.136  0.001  0.063 -0.    -0.     0.181  0.972]


-3.5[ 0.117 -0.015 -0.992  0.009  0.     0.    -0.047  0.009]




<class 'numpy.ndarray'>
3.5[ 0.361 -0.013 -0.232  0.884  0.     0.     0.182 -0.046]
2.5[-0.003 -0.086 -0.225 -0.059 -0.    -0.    -0.237 -0.939]
1.5[ 0.  0. -0. -0. -0.  1. -0.  0.]
0.5[ 0.928 -0.035  0.085 -0.322  0.    -0.    -0.158  0.04 ]
-0.5[ 0.035  0.928  0.322  0.085 -0.     0.    -0.04  -0.158]
-1.5[ 0.  0.  0.  0.  1.  0. -0. -0.]
-2.5[ 0.086 -0.003  0.059

In [26]:
res[1][:,0]

array([-1.51230072e-02, -1.35951024e-01, -4.16333634e-17, -1.26028985e-01,
       -9.75396061e-01,  0.00000000e+00,  1.75659614e-02,  1.17043882e-01])

In [112]:
#%timeit err2([1,-0.0615,0.0011,0.005,0.315,0.037])
%timeit eigenfunc(Matrix(B))
#timeit Sqw_optim(eigensystem)
#err2([-2.105e+00, -2.000e-03, -1.000e-03,  2.900e-02, -6.950e-01, -1.700e-02])

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

...

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

In [47]:

""" 
def err(B,r=False):
    #B = [1.135,-0.0615,0.0011,0.005,0.315,0.037]
    

    
    
    rms = ()#+ mean_absolute_error(a,actual))
    #return [rms,B,a] 
    if r==False:
        return rms
    else:
        return [rms,B,a,temp]
        """  

' \ndef err(B,r=False):\n    #B = [1.135,-0.0615,0.0011,0.005,0.315,0.037]\n    \n\n    \n    \n    rms = ()#+ mean_absolute_error(a,actual))\n    #return [rms,B,a] \n    if r==False:\n        return rms\n    else:\n        return [rms,B,a,temp]\n        '

In [109]:
err2([1.135,-0.0615,0.0011,0.005,0.315,0.037])

%timeit err2(np.ones(6))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
2.17 ms ± 190 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [106]:
from scipy.optimize import minimize 

paper = [1.270,-0.0372,0.00025,0.0024,0.275,0.0023]

#b =[1.135,-0.0615,0.0011,0.005,0.315,0.037] +  0.5*np.ones(6)
b = paper
result = minimize(err2, b, method='Powell')


<class 'numpy.ndarray'>
12.7
<class 'numpy.ndarray'>
12.7
<class 'numpy.ndarray'>
22.7
<class 'numpy.ndarray'>
-3.4803399999999995
<class 'numpy.ndarray'>
3.6515326426241588
<class 'numpy.ndarray'>
3.6515326426241588
<class 'numpy.ndarray'>
7.10773952525158
<class 'numpy.ndarray'>
9.243792889749335
<class 'numpy.ndarray'>
5.3985780270123165
<class 'numpy.ndarray'>
7.923639284675329
<class 'numpy.ndarray'>
6.454897944415121
<class 'numpy.ndarray'>
7.419385492759631
<class 'numpy.ndarray'>
6.858376237985801
<class 'numpy.ndarray'>
7.22677768887676
<class 'numpy.ndarray'>
7.012491227867819
<class 'numpy.ndarray'>
7.1636621300990635
<class 'numpy.ndarray'>
7.1636621300990635
<class 'numpy.ndarray'>
7.1636621300990635
<class 'numpy.ndarray'>
7.1636621300990635
<class 'numpy.ndarray'>
7.1636621300990635
<class 'numpy.ndarray'>
7.1636621300990635
<class 'numpy.ndarray'>
7.1636621300990635
<class 'numpy.ndarray'>
7.1636621300990635
<class 'numpy.ndarray'>
7.1636621300990635
<class 'numpy.ndarr

<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.465766769361409
<class 'numpy.ndarray'>
8.46576676

<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.737865308490258
<class 'numpy.ndarray'>
8.73786530

<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.939493197037784
<class 'numpy.ndarray'>
8.592133850013347
<class 'numpy.ndarray'>
8.806813733354563
<class 'numpy.ndarray'>
8.724813317034252
<class 'numpy.ndarray'>
8.85749277

9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.ndarray'>
9.109426559533752
<class 'numpy.nd

<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.109425999924248
<class 'numpy.ndarray'>
9.10942599

<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.109425268173338
<class 'numpy.ndarray'>
9.33807568

In [105]:
clear_output()
print(np.around(result.x,150))#, result.success, result.message)
print('\n Error')
print(err2(result.x), err2([1.135,-0.0615,0.0011,0.005,0.315,0.037]))
print('\n Error Parameters')

print(np.sqrt(mean_squared_error(result.x,[1.135,-0.0615,0.0011,0.005,0.315,0.037])))


res = eigenfunc(Matrix(result.x))
t= res[0] 
t =t - min(t)
print("Eigen Energies")
print(np.round(t,4))
print("Relative Intensity \n")
print(Sqw2(result.x))


print("\n")
#print(np.around(res[1][:,0],3),res[0][0])
print("\n")


for i in range(8):
    print(np.around(res[1][i,:],3))
    print("\n")


#[1.135,-0.0615,0.0011,0.005,0.315,0.037]

[ 1.13508013 -0.03482644  0.00152346 -0.00955713  0.72114096 -0.04690294]

 Error
<class 'numpy.ndarray'>
11.350801270864082
<class 'numpy.ndarray'>
11.35
0.0009283402173674052 0.0

 Error Parameters
0.1697614784551381
<class 'numpy.ndarray'>
Eigen Energies
[  0.       0.      76.7987  76.7987  82.1193  82.1193 116.2431 116.2431]
Relative Intensity 

<class 'numpy.ndarray'>
[0.19042631 0.28437459]




[ 0.011 -0.    -0.015 -0.959  0.    -0.    -0.281  0.017]


[ 0.004  0.453  0.256 -0.004 -0.     0.    -0.051 -0.852]


[ 0.     0.     0.    -0.     0.997 -0.075  0.     0.   ]


[ 0.891 -0.008 -0.002 -0.118 -0.    -0.     0.437 -0.026]


[ 0.008  0.891 -0.118  0.002 -0.     0.     0.026  0.437]


[-0.    -0.    -0.     0.     0.075  0.997 -0.    -0.   ]


[-0.453  0.004 -0.004 -0.256 -0.     0.     0.852 -0.051]


[-0.    -0.011  0.959 -0.015 -0.     0.     0.017  0.281]




In [81]:
np.linalg.norm(result.x)

2.1290552150415487

In [65]:
from scipy.optimize import basinhopping 

paper = [1.270,-0.0372,0.00025,0.0024,0.275,0.0023]

#b =[1.135,-0.0615,0.0011,0.005,0.315,0.037] +  0.5*np.ones(6)
b = paper
result = basinhopping(err2, b, niter = 5, minimizer_kwargs ={ 'method' : 'Powell'})

<class 'numpy.ndarray'>
1217702.7771193068 891.6259868596574
<class 'numpy.ndarray'>
1217702.7771193068 891.6259868596574
<class 'numpy.ndarray'>
102451081.2205038 376.74224785841585
<class 'numpy.ndarray'>
7357.18741993673 1934.9317318167032
<class 'numpy.ndarray'>
93.80196812242714 1763.8508060829536
<class 'numpy.ndarray'>
93.80196812242714 1763.8508060829536
<class 'numpy.ndarray'>
215.00881326569436 1431.294990647102
<class 'numpy.ndarray'>
414892.1490619623 1211.2701303851577
<class 'numpy.ndarray'>
138.57802937833063 1599.582089462358
<class 'numpy.ndarray'>
259783.39394679267 1346.3940823395305
<class 'numpy.ndarray'>
185.4892524180704 1495.675540337359
<class 'numpy.ndarray'>
215017.5260364347 1399.1522018294559
<class 'numpy.ndarray'>
203.81603069029106 1455.8546454513407
<class 'numpy.ndarray'>
199708.61386828928 1419.412695551278
<class 'numpy.ndarray'>
210.75057575584222 1440.6690368171508
<class 'numpy.ndarray'>
217.49590303178582 1425.800137018911
<class 'numpy.ndarray'>

72.71300128900029 13.549102503168303
<class 'numpy.ndarray'>
72.68901570624368 13.517929635956769
<class 'numpy.ndarray'>
72.50313873125691 13.384665129703476
<class 'numpy.ndarray'>
72.61903882715444 13.445771695287268
<class 'numpy.ndarray'>
72.60519864760843 13.43472539027023
<class 'numpy.ndarray'>
72.63283077457784 13.457824082767683
<class 'numpy.ndarray'>
72.65438697816515 13.478775964586257
<class 'numpy.ndarray'>
72.66770485373544 13.493025295551522
<class 'numpy.ndarray'>
72.65438697816515 13.478775964586257
<class 'numpy.ndarray'>
35107361913.283295 91547456.40091348
<class 'numpy.ndarray'>
1512.7018377115241 469592984.64463615
<class 'numpy.ndarray'>
72.65438697816515 13.478775964586257
<class 'numpy.ndarray'>
1512.7461911832863 67519453.49701966
<class 'numpy.ndarray'>
63058125310.59472 13232271.444940934
<class 'numpy.ndarray'>
1512.974731430824 14597081.6184225
<class 'numpy.ndarray'>
11034477515.20595 1884381.1138291003
<class 'numpy.ndarray'>
1515.0001747453762 1958441

39282.76338188039 1185673.0934756876
<class 'numpy.ndarray'>
38478.312000243845 3160103.072442653
<class 'numpy.ndarray'>
5.0701487372228025 11.873280656391469
<class 'numpy.ndarray'>
40697.38646542978 375888.79653138993
<class 'numpy.ndarray'>
42258.123618548976 137787.17344719538
<class 'numpy.ndarray'>
1725.5961861113735 349.41793766417493
<class 'numpy.ndarray'>
426.975632356674 470.55612383890053
<class 'numpy.ndarray'>
53.267493376244445 8.111913889495789
<class 'numpy.ndarray'>
4.170654717342639 22.00605724921558
<class 'numpy.ndarray'>
5.340968708433546 11.764316525249395
<class 'numpy.ndarray'>
2.9642790285152727 13.01988499760776
<class 'numpy.ndarray'>
4.16525595152087 12.287617206334527
<class 'numpy.ndarray'>
4.5285838497381325 12.110803059067612
<class 'numpy.ndarray'>
4.523382586543213 12.113226810898732
<class 'numpy.ndarray'>
4.730427217649412 12.018946642406226
<class 'numpy.ndarray'>
4.85827063306278 11.962877026065359
<class 'numpy.ndarray'>
4.652599371961608 12.053

1.079701016477371 16.04848535577669
<class 'numpy.ndarray'>
11.060044703111855 11.536011503439818
<class 'numpy.ndarray'>
3.99934704588771 8.312694943890262
<class 'numpy.ndarray'>
5.078976194290709 8.0897353533136
<class 'numpy.ndarray'>
5.481490342850904 8.115784679106504
<class 'numpy.ndarray'>
5.484709525760653 8.116187652622246
<class 'numpy.ndarray'>
5.478272383909268 8.11538479700621
<class 'numpy.ndarray'>
5.481490342850904 8.115784679106504
<class 'numpy.ndarray'>
39241.29034265342 1185893.0119630762
<class 'numpy.ndarray'>
38457.22087102316 3159892.2390145767
<class 'numpy.ndarray'>
5.481490342850904 8.115784679106504
<class 'numpy.ndarray'>
40637.08943160268 375810.66178977373
<class 'numpy.ndarray'>
42086.950189932984 137898.0746151015
<class 'numpy.ndarray'>
1723.2466177676267 347.593775003034
<class 'numpy.ndarray'>
480.1249635272568 472.1312424801933
<class 'numpy.ndarray'>
59.0094596366575 5.588708957520151
<class 'numpy.ndarray'>
7.648003447076175 18.70286906867282
<cl

31690.234504194344 92.8802545079462
<class 'numpy.ndarray'>
9.655987947203803 10.891727927665933
<class 'numpy.ndarray'>
1.4791110648110786 0.7245896488178093
<class 'numpy.ndarray'>
2.0146149634792283 0.552688126016984
<class 'numpy.ndarray'>
1.5524283509660401 0.6827711998425866
<class 'numpy.ndarray'>
1.6059675379032097 0.6563188930700574
<class 'numpy.ndarray'>
1.5359350701903491 0.6916030333781626
<class 'numpy.ndarray'>
1.5253820204812885 0.6974268094284621
<class 'numpy.ndarray'>
1.5326233418128132 0.6934160273952716
<class 'numpy.ndarray'>
1.5293480292603654 0.6952222110836391
<class 'numpy.ndarray'>
1.5326233418128132 0.6934160273952716
<class 'numpy.ndarray'>
7.416658708516441 5.380671634093602
<class 'numpy.ndarray'>
6.012157078132429 22.867868598349236
<class 'numpy.ndarray'>
1.5326233418128132 0.6934160273952716
<class 'numpy.ndarray'>
2.1152146478792733 5.121466347032596
<class 'numpy.ndarray'>
2.5149448378069303 0.6847016629044722
<class 'numpy.ndarray'>
1.64605445856379

...

7.034734105734194 13.646710522378152
<class 'numpy.ndarray'>
10425.407791705353 90448745.39572155
<class 'numpy.ndarray'>
4839.622837645859 470420112.1053388
<class 'numpy.ndarray'>
7.034734105734194 13.646710522378152
<class 'numpy.ndarray'>
4838.877364312713 67834195.08848535
<class 'numpy.ndarray'>
10436.24400866614 12817570.71021115
<class 'numpy.ndarray'>
55.09005962795678 432.42239396171107
<class 'numpy.ndarray'>
10511.020493544875 526891.5303215273
<class 'numpy.ndarray'>
10800.714247549677 53202.15732583312
<class 'numpy.ndarray'>
8479.666852018992 948.3740021423782
<class 'numpy.ndarray'>
168651.01793998477 8.176477621493023
<class 'numpy.ndarray'>
19.977356852324405 87.02415271063184
<class 'numpy.ndarray'>
10.850801233755462 29.108644011667373
<class 'numpy.ndarray'>
192323.4764812931 7.834535681812522
<class 'numpy.ndarray'>
8.304385210650615 17.723426244473806
<class 'numpy.ndarray'>
6.073237093968586 11.875429249756179
<class 'numpy.ndarray'>
6.602773553570879 12.6606180

0.8691389725866048 0.2759592184622355
<class 'numpy.ndarray'>
0.6654986256318424 0.3084316559039184
<class 'numpy.ndarray'>
0.6587019603076525 0.31052154039680946
<class 'numpy.ndarray'>
0.6519801040089745 0.3126950294088365
<class 'numpy.ndarray'>
0.6587019603076525 0.31052154039680946
<class 'numpy.ndarray'>
4.78908750746978 2.775544182784313
<class 'numpy.ndarray'>
8.400627303035645 10.152514792996607
<class 'numpy.ndarray'>
0.6587019603076525 0.31052154039680946
<class 'numpy.ndarray'>
2.2582757682555563 1.7552958362905096
<class 'numpy.ndarray'>
0.9164254255728799 0.622969309505588
<class 'numpy.ndarray'>
0.6193965708971971 0.3036159099135765
<class 'numpy.ndarray'>
0.6139061742193328 0.3146547760172579
<class 'numpy.ndarray'>
0.6185437907941377 0.32922997843681945
<class 'numpy.ndarray'>
0.6181578765784782 0.3284073725260808
<class 'numpy.ndarray'>
0.617792213919399 0.32760545797889673
<class 'numpy.ndarray'>
8.425771043735026 3.7989812882077123
<class 'numpy.ndarray'>
0.61815787

0.3166145944416095 0.021078871324625856
<class 'numpy.ndarray'>
0.335071732039667 0.01679753958348879
<class 'numpy.ndarray'>
0.32415806949521714 0.018816941928266863
<class 'numpy.ndarray'>
0.32920946072356383 0.017756713054527237
<class 'numpy.ndarray'>
0.32589529359264774 0.018422243873657228
<class 'numpy.ndarray'>
0.32627544817556053 0.018340401778488957
<class 'numpy.ndarray'>
0.32594071289588455 0.01841238430092315
<class 'numpy.ndarray'>
0.32591769424568356 0.01841737837663689
<class 'numpy.ndarray'>
0.32592765090576936 0.018415217499053282
<class 'numpy.ndarray'>
0.3259333370213262 0.018413983931288798
<class 'numpy.ndarray'>
0.32593080486581666 0.01841453322370579
<class 'numpy.ndarray'>
0.3259323698036895 0.018414193738552225
<class 'numpy.ndarray'>
0.32593296757410817 0.018414064069981726
<class 'numpy.ndarray'>
0.32593351607515747 0.018413945092336664
<class 'numpy.ndarray'>
0.325933626736918 0.018413921088621405
<class 'numpy.ndarray'>
0.32593369512982256 0.01841390625354

0.3263892954124067 0.01831532926127315
<class 'numpy.ndarray'>
0.3259989085627196 0.018398895232377054
<class 'numpy.ndarray'>
0.32593078303915013 0.0184136443672545
<class 'numpy.ndarray'>
0.3259188342887688 0.018416236420130516
<class 'numpy.ndarray'>
0.32592491130903234 0.018414917934151738
<class 'numpy.ndarray'>
0.3259268267178853 0.01841450244476852
<class 'numpy.ndarray'>
0.32592685371425 0.018414496589019995
<class 'numpy.ndarray'>
0.32592680840025895 0.01841450641802962
<class 'numpy.ndarray'>
0.3259268370295796 0.018414500208072747
<class 'numpy.ndarray'>
0.32592684340256345 0.018414498825716837
<class 'numpy.ndarray'>
0.32592683309088655 0.018414501062407454
<class 'numpy.ndarray'>
0.325926839463852 0.018414499680057202
<class 'numpy.ndarray'>
0.3259268409683216 0.018414499353731368
<class 'numpy.ndarray'>
0.325926838534065 0.01841449988174111
<class 'numpy.ndarray'>
0.3259268399882892 0.01841449956630501
<class 'numpy.ndarray'>
0.325926839463852 0.018414499680057202
<class 

<class 'numpy.ndarray'>
0.3253672688435669 0.018430469931074518
<class 'numpy.ndarray'>
0.32653660299552845 0.01840047133990659
<class 'numpy.ndarray'>
0.3261597216555402 0.018408726737231852
<class 'numpy.ndarray'>
0.3257353692075793 0.018419630585559676
<class 'numpy.ndarray'>
0.3260157886340844 0.0184122341266749
<class 'numpy.ndarray'>
0.32586229684284973 0.018416190477360244
<class 'numpy.ndarray'>
0.32596081445207503 0.018413625490651866
<class 'numpy.ndarray'>
0.32590550040461924 0.018415054326832905
<class 'numpy.ndarray'>
0.3259398166711216 0.01841416448136384
<class 'numpy.ndarray'>
0.3259199541207206 0.01841467817286798
<class 'numpy.ndarray'>
0.3259317963038624 0.018414371457111483
<class 'numpy.ndarray'>
0.32592467724429613 0.018414555683550778
<class 'numpy.ndarray'>
0.3259287328066361 0.018414450675245473
<class 'numpy.ndarray'>
0.3259261681764368 0.01841451706161459
<class 'numpy.ndarray'>
0.3259275626563531 0.018414480957273587
<class 'numpy.ndarray'>
0.325927115698971

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


0.0005779993011646483

In [89]:
def Jplus(bra,ket):
    j=3.5
    a= np.zeros(8)
    for i in range(7):
        m = i - 3.5
        if (i<7):
            a[i+1] = np.sqrt((j*(j+1) - (m)*(m+1)))*ket[i]
    print(a)
    return(np.dot(bra,a))


def Jzzz(ket):
    
    a= np.zeros(8)
    for i in range(7):
        m = i - 3.5
        a[i] = m*ket[i]
    
    return(np.dot(ket,a))


a = np.zeros(8)
b = np.zeros(8)

a[4] = -0.9
a[7] = 0.3

b[3] = -0.9
b[0] = -0.3

#Jplus(a,b)
Jzzz(b)

-0.72

In [90]:
p = eigenfunc(Matrix([1.135,-0.0615,0.0011,0.005,0.315,0.037]))[1]

(2*Jzzz(p[:,0]))/Jplus(p[:,1],p[:,0])


<class 'numpy.ndarray'>
[ 0.00000000e+00  9.54164507e-01 -1.11684089e-02  1.07496882e-16
  3.71177598e+00  1.34074032e-01  0.00000000e+00  2.28650645e-01]


-0.5284565583500347

In [91]:
(2*Jzzz(res[1][:,0]))/Jplus(res[1][:,0],res[1][:,1])

[ 0.         -0.309669    0.06085028  0.         -3.90158424  0.48810816
  0.          0.3596926 ]


0.2167321998110483

In [69]:
p[:,1],p[:,0]

(array([-1.34540027e-02, -8.64218204e-02,  1.68268177e-16, -3.46177662e-02,
         9.27943996e-01,  0.00000000e+00, -3.22404195e-03, -3.60640285e-01]),
 array([ 3.60640285e-01, -3.22404195e-03,  2.77555756e-17,  9.27943996e-01,
         3.46177662e-02,  0.00000000e+00,  8.64218204e-02, -1.34540027e-02]))

In [82]:
0.9*0.9*0.5+0.3*0.3*3.5

0.72